# Demo Script for Milestone 10.15

## 0 Prepare task

### 0.1 prepare planning scene

#### Run shared detector on bash
```bash
python3 /home/kiro-ros/Projects/rnb-planning/src/scripts/demo_202107/demo_utils/shared_detector.py
```

#### Check and request ip setting from mobile udp client (robot-side)

#### 0.1.1 Set parameters and create planning scene

In [6]:
raw_input("==================================================\n=========== Start IndySDK and press Enter ========\n")

=========== Start IndySDK and press Enter ========



''

In [8]:
raw_input("==================================================\n========= Connect to Conty and reset robot =======\n")

========= Connect to Conty and reset robot =======



''

In [1]:
import numpy as np

CONNECT_INDY = False
CONNECT_MOBILE = False

CONNECT_TASK_PLANNER = False
VISUALIZE = False
VERBOSE = False
PLANNING_MULTIPROC = True
N_AGENTS = 10
NODE_TRIAL_MAX = N_AGENTS * 2
MAX_SOL_NUM = 5
BASE_COST_CUT = 110

TIMEOUT_MOTION = 0.5
TIMEOUT_FULL = 5

ROS_MASTER_ON_MOBILE = False
# Tool dimensions
TOOL_DIM = [0.24, 0.35]
TOOL_CENTER_OFF = [0, -0.035, 0]
TOOL_THICKNESS = 0.11 + 0.05

INDY_BASE_OFFSET = (0.172,0,0.439)
INDY_BASE_RPY = (0,0,0)
TOOL_NAME = "brush_face"
WALL_THICKNESS = 0.01
CLEARANCE = 0.001
WS_HEIGHT = 1.6

COL_COLOR = (1,1,1,0.2)

USE_SDK = True

IP_CUR = "192.168.0.10"# get_ip_address()
MOBILE_IP = "192.168.0.102"
INDY_IP = "192.168.0.3"

print("Current PC IP: {}".format(IP_CUR))
print("Mobile ROB IP: {}".format(MOBILE_IP))

Current PC IP: 192.168.0.10
Mobile ROB IP: 192.168.0.102


In [2]:
import os
import sys
sys.path.append(os.path.join(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src')))
sys.path.append(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src/scripts/demo_202107'))

from pkg.global_config import RNB_PLANNING_DIR
from pkg.utils.utils import *    
from pkg.utils.rotation_utils import *
from pkg.controller.combined_robot import *
from demo_utils.area_select import *
from pkg.detector.aruco.marker_config import get_aruco_map
aruco_map = get_aruco_map()

from pkg.detector.multiICP.multiICP import *
from pkg.detector.detector_interface import DetectionLevel
from pkg.detector.multiICP.config import *

from pkg.geometry.builder.scene_builder import SceneBuilder
from demo_utils.environment import *
from demo_utils.area_select import DATASET_DIR, SweepDirections
from demo_utils.demo_config import *
from demo_utils.detection_util import *

from pkg.utils.shared_function import *
clear_channels_on("kiromobilemap")

if not CONNECT_INDY:
    indy_7dof_client.kiro_tool.OFFLINE_MODE = True
kiro_udp_client.KIRO_UDP_OFFLINE_DEBUG = not CONNECT_MOBILE

mobile_config = RobotConfig(0, RobotType.kmb, ((0,0,0), (0,0,0)),
                "{}/{}".format(MOBILE_IP, IP_CUR))
robot_config = RobotConfig(1, RobotType.indy7kiro, 
                           (INDY_BASE_OFFSET, INDY_BASE_RPY),
                INDY_IP, root_on="kmb0_platform", 
                           specs={"no_sdk":True} if not USE_SDK else {})
ROBOT_TYPE = robot_config.type
MOBILE_NAME = mobile_config.get_indexed_name()
ROBOT_NAME = robot_config.get_indexed_name()
crob = CombinedRobot(robots_on_scene=[mobile_config, robot_config]
              , connection_list=[True, CONNECT_INDY])

s_builder = SceneBuilder(None)
SceneBuilder.autostart_roscore = not ROS_MASTER_ON_MOBILE
gscene = s_builder.create_gscene(crob)

gtems = s_builder.add_robot_geometries(
    color=COL_COLOR, display=True, collision=True)

gscene.set_workspace_boundary(
    -0.7, 1.55, -0.7, 2.5, -CLEARANCE, WS_HEIGHT, thickness=1.0)


from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

ROBOT_BASE = pscene.robot_chain_dict[ROBOT_NAME]['link_names'][0]
TIP_LINK = pscene.robot_chain_dict[ROBOT_NAME]["tip_link"]
CAM_LINK = TIP_LINK.replace("tcp", "link6")
MOBILE_BASE = pscene.robot_chain_dict[MOBILE_NAME]["tip_link"]
HOLD_LINK = MOBILE_BASE

viewpoint = add_cam(gscene, tool_link=CAM_LINK, center=(-0.0785, 0, 0.073))

add_brush(gscene, face_name=TOOL_NAME, tool_link=TIP_LINK,
          thickness=TOOL_THICKNESS, tool_dim=TOOL_DIM,
          col_color=COL_COLOR, center_off=TOOL_CENTER_OFF)

HOME_POSE = crob.home_pose
HOME_DICT = list2dict(HOME_POSE, gscene.joint_names)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

# Set planner
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.filtering.grasp_filter import GraspChecker
mplan = MoveitPlanner(pscene, enable_dual=False, 
                      incremental_constraint_motion=True)
mplan.motion_filters = [GraspChecker(pscene)]
mplan.update_gscene()
gcheck = GraspChecker(pscene)
mplan.motion_filters = [gcheck]

mplan.reset_PRQdict(enable_PRQ=0.5, radii=5e-2)
for tip_dir, SWEEP_AXIS in [
    (SweepDirections.front, "X"), (SweepDirections.up, "Z"), (SweepDirections.down, "Z")]:
    filename = SweepDirections.get_file_name(ROBOT_TYPE, tip_dir.name+SWEEP_AXIS)+"-PRQ.pkl"
    PRQ_PATH = os.path.join(SWEEP_DATA_DIR, filename)
    try:
        Pos_Rotvec_Qlist_dict = load_pickle(PRQ_PATH)
        mplan.register_PRQ(ROBOT_NAME, Pos_Rotvec_Qlist_dict, decimal=2)
        print("Loaded: {}".format(filename))
    except:
        print("File not exist: {}".format(filename))
        continue

from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene, node_trial_max=NODE_TRIAL_MAX)
tplan.prepare()
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

# Register binders
from pkg.planning.constraint.constraint_actor import VacuumTool, \
    Gripper2Tool, PlacePlane, SweepFramer

brush_face = pscene.create_binder(
    bname=TOOL_NAME, gname=TOOL_NAME, _type=SweepFramer, 
    point=(0,0, -gscene.NAME_DICT['brush_face'].dims[2]/2-CLEARANCE), 
    rpy=(0,0,0))

gscene.create_safe(
    gtype=GEOTYPE.BOX, name="floor_box", link_name="base_link",
    dims=(15,15,0.4), center=(0,0,0), rpy=(0,0,0), 
    color=(1, 1, 1, 0.1), display=True, collision=False, fixed=True)

gscene.add_highlight_axis("hl", "base_coord", T=np.identity(4), dims=(0.5,0.1,0.1))

kmb = crob.robot_dict["kmb0"]
indy = crob.robot_dict["indy1"]
mobile_box = gscene.NAME_DICT['kmb0_platform_Box_2']
crob.simulator.set_gscene(gscene)
# indy.ktool.led_on()
# indy.ktool.led_off()
# time.sleep(2)
# indy.ktool.send_degree(-45)

if CONNECT_MOBILE:
    assert np.sum(np.abs(get_xyzw_cur()))>1e-4, "UDP Server not connected"
    
if CONNECT_TASK_PLANNER:
    from demo_proto.DisinfectionOperationServicer import serve_on_thread
    servicer = serve_on_thread()

connection command:
kmb0: True
indy1: False
==== Kiro Tool connected to /dev/ttyUSB0 (115200) ====
[KTOOL] initialize
[KTOOL] enable
[KTOOL] op_init
[KTOOL] led_off
Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker
Loaded: indy7kiro-frontX-PRQ.pkl
Loaded: indy7kiro-upZ-PRQ.pkl
Loaded: indy7kiro-downZ-PRQ.pkl


In [3]:
if CONNECT_INDY:
    with indy:
        stat = indy.get_robot_status()
        if stat['error'] or not stat['ready']:
            raise(RuntimeError("Indy not ready"))

#### 0.1.2 Load environment map

In [4]:
from demo_utils.ros_map_utils import KiroMobileMap
kmm = KiroMobileMap(MOBILE_IP, IP_CUR, CONNECT_MOBILE)
            
VALID_BOX_SCALE = 0.7
VALID_SCORE_CUT = 50
kmb.coster = (lambda Q: 
                  np.max(
                      kmm.get_box_costs(mobile_box, Q, kmm.T_bi, kmm.cost_im, kmm.resolution, 
                                        scale=VALID_BOX_SCALE)))
kmb.cost_cut = VALID_SCORE_CUT
kmb.gscene = gscene

kmm.init_node(timeout=10)

In [5]:
pole_pt_list, pole_res = kmm.update_map(gscene, crob, MOBILE_BASE, timeout=100)
if not CONNECT_MOBILE:
    crob.joint_move_make_sure(kmm.Q_map)
    
if CONNECT_INDY:
    indy.move_joint_s_curve(crob.home_pose[6:], N_div=500)
else:
    indy.joint_move_make_sure(crob.home_pose[6:])

Qcur = crob.get_real_robot_pose()
crob.joint_move_make_sure(Qcur)

[WARN] Vertices for mesh should be have center point (0,0,0). Auto adjusting.
End up at=[ 0.049  0.174  0.024] (3.47e-18 / 3.47e-18)
End up at=[ 0.049  0.174  0.024] (3.47e-18 / 3.47e-18)


In [6]:
# pole_list = kmm.add_pixel_poles("obs_pt", gscene, pole_pt_list, pole_res, height=WS_HEIGHT)
# gcheck.ignore_always = pole_list

# gscene.update_markers_all() 

## 1. Prepare scene

In [7]:
HEIGHT_OFFSET = 0.02
TABLE_DIMS = (0.60,1.2, 0.71)
TABLE_LOC = (1.17,0.05,TABLE_DIMS[2]/2+HEIGHT_OFFSET)
TABLE_ANG = np.pi - np.deg2rad(0)
TABLE_MARGIN_I = 0.03
TABLE_MARGIN_O = 0.1
CLEARANCE = 0.002
TABLE_COLOR = (0.8,0.8,0.8,1)
SURF_COLOR = (0,1,0,0.4)
COL_COLOR = (1,1,1,0.2)

gscene.create_safe(GEOTYPE.BOX, "table", link_name="base_link",
                   dims=TABLE_DIMS, center=TABLE_LOC, rpy=Rot2rpy(Rot_axis(3, TABLE_ANG)),
                   color=TABLE_COLOR, collision=False
                  )
gscene.create_safe(GEOTYPE.BOX, "table_col", link_name="base_link",
                   dims=np.add(TABLE_DIMS, 
                               (TABLE_MARGIN_O*2, TABLE_MARGIN_O*2, 0)), 
                   center=(0,0,0), rpy=(0,0,0), color=COL_COLOR, 
                   collision=True, parent="table"
                  )
table_surf = gscene.create_safe(GEOTYPE.BOX, "table_surf", link_name="base_link",
                   dims=np.subtract(TABLE_DIMS[:2]+(CLEARANCE,), 
                                    (TABLE_MARGIN_I*2, TABLE_MARGIN_I*2, 0)), 
                   center=(0,0,(TABLE_DIMS[2]+CLEARANCE)/2), 
                   rpy=(0,0,0), color=SURF_COLOR, parent="table",
                   collision=False
                  )

In [8]:
SOFA_DIMS = (0.50,0.98, 0.45)
SOFA_LOC = (1.15,1.4,SOFA_DIMS[2]/2+HEIGHT_OFFSET)
SOFA_SIDE_THIC = 0.2
SOFA_SIDE_HEIT = 0.27
SOFA_ANG = np.pi + np.deg2rad(0)

SOFA_MARGIN_O = 0.05
SOFA_MARGIN_Ix = 0.2 # 앞뒤
SOFA_MARGIN_Iy = 0.15  # 좌우
SOFA_MARGIN_Iz = 0.05 # 아래
SOFA_UV_DIST = 0.15

SOFA_COLOR = (0.4,0.4,0.4,1)

sofa = gscene.create_safe(GEOTYPE.BOX, "sofa", link_name="base_link",
                   dims=SOFA_DIMS, center=SOFA_LOC, 
                   rpy=Rot2rpy(Rot_axis(3, SOFA_ANG)),
                   color=SOFA_COLOR, collision=False
                  )
gscene.create_safe(GEOTYPE.BOX, "sofa_col", link_name="base_link",
                   dims=np.add(SOFA_DIMS, SOFA_MARGIN_O*2), 
                   center=(0,0,0), rpy=(0,0,0),
                   color=COL_COLOR, collision=True, parent="sofa"
                  )
sofa_left = gscene.create_safe(GEOTYPE.BOX, "sofa_left", link_name="base_link",
                   dims=(SOFA_DIMS[0], SOFA_SIDE_THIC, SOFA_DIMS[2]+SOFA_SIDE_HEIT), 
                   center=(0,(SOFA_DIMS[1]+SOFA_SIDE_THIC)/2, SOFA_SIDE_HEIT/2), 
                   rpy=(0,0,0), color=SOFA_COLOR, collision=False, parent="sofa"
                  )
gscene.create_safe(GEOTYPE.BOX, "sofa_left_col", link_name="base_link",
                   dims=np.add(sofa_left.dims, SOFA_MARGIN_O*2), 
                   center=(0,0,0), rpy=(0,0,0),
                   color=COL_COLOR, collision=True, parent="sofa_left"
                  )
sofa_right = gscene.create_safe(GEOTYPE.BOX, "sofa_right", link_name="base_link",
                   dims=(SOFA_DIMS[0], SOFA_SIDE_THIC, SOFA_DIMS[2]+SOFA_SIDE_HEIT), 
                   center=(0,-(SOFA_DIMS[1]+SOFA_SIDE_THIC)/2, SOFA_SIDE_HEIT/2), 
                   rpy=(0,0,0), color=SOFA_COLOR, collision=False, parent="sofa"
                  )
gscene.create_safe(GEOTYPE.BOX, "sofa_right_col", link_name="base_link",
                   dims=np.add(sofa_right.dims, SOFA_MARGIN_O*2), 
                   center=(0,0,0), rpy=(0,0,0),
                   color=COL_COLOR, collision=True, parent="sofa_right"
                  )
sofa_back = gscene.create_safe(GEOTYPE.BOX, "sofa_back", link_name="base_link",
                   dims=(SOFA_SIDE_THIC, SOFA_DIMS[1]+SOFA_SIDE_THIC*2, SOFA_DIMS[2]+SOFA_SIDE_HEIT), 
#                    dims=(SOFA_SIDE_THIC, SOFA_DIMS[1], SOFA_DIMS[2]+SOFA_SIDE_HEIT), 
                   center=(-(SOFA_DIMS[0]+SOFA_SIDE_THIC)/2, 0, SOFA_SIDE_HEIT/2), 
                   rpy=(0,0,0), color=SOFA_COLOR, collision=False, parent="sofa"
                  )
gscene.create_safe(GEOTYPE.BOX, "sofa_back_col", link_name="base_link",
                   dims=np.add(sofa_back.dims, SOFA_MARGIN_O*2), 
                   center=(0,0,0), rpy=(0,0,0),
                   color=COL_COLOR, collision=True, parent="sofa_back"
                  )

sofa_surf = gscene.create_safe(GEOTYPE.BOX, "sofa_surf", link_name="base_link",
                               dims=tuple(np.subtract(SOFA_DIMS[:2], (SOFA_MARGIN_Ix, SOFA_MARGIN_Iy*2)))+(CLEARANCE,), # (방석 앞뒤/좌우 마진)
                               center=(SOFA_MARGIN_Ix/2,0,SOFA_DIMS[2]/2+SOFA_UV_DIST), collision=False, 
                               rpy=(0,0,0), color=SURF_COLOR, parent="sofa"
                              )

sofa_back_surf = gscene.create_safe(GEOTYPE.BOX, "sofa_back_surf", link_name="base_link",
                               dims=sofa_back.dims[:2]+(CLEARANCE,), collision=False, 
                               center=(SOFA_MARGIN_Ix,0,sofa_back.dims[2]/2+SOFA_MARGIN_O+CLEARANCE), #  (등위 앞쪽 오프셋)
                               rpy=(0,0,0), color=SURF_COLOR, parent="sofa_back"
                              )

sofa_left_surf = gscene.create_safe(GEOTYPE.BOX, "sofa_left_surf", link_name="base_link",
                               dims=sofa_left.dims[:2]+(CLEARANCE,), collision=False, 
                               center=(0,0,sofa_left.dims[2]/2+SOFA_UV_DIST), 
                               rpy=(0,0,0), color=SURF_COLOR, parent="sofa_left"
                              )

sofa_right_surf = gscene.create_safe(GEOTYPE.BOX, "sofa_right_surf", link_name="base_link",
                               dims=sofa_right.dims[:2]+(CLEARANCE,), collision=False, 
                               center=(0,0,sofa_right.dims[2]/2+SOFA_UV_DIST), 
                               rpy=(0,0,0), color=SURF_COLOR, parent="sofa_right"
                              )

sofa_front = gscene.create_safe(GEOTYPE.BOX, "sofa_front", link_name="base_link",
                               dims=(sofa.dims[2], sofa.dims[1])+(CLEARANCE,), 
                               center=(sofa.dims[0]/2+SOFA_UV_DIST,0,SOFA_MARGIN_Iz), collision=False,  #  (다리앞 바닥 오프셋)
                               rpy=(0,np.pi/2,0), color=SURF_COLOR, parent="sofa"
                              )

sofa_left_front = gscene.create_safe(GEOTYPE.BOX, "sofa_left_front", link_name="base_link",
                               dims=(sofa_left.dims[2]-SOFA_MARGIN_Iz, sofa_left.dims[1])+(CLEARANCE,), 
                               center=(sofa_left.dims[0]/2+SOFA_UV_DIST,0,+SOFA_MARGIN_Iz/2), collision=False, # (왼쪽앞 바닥 오프셋)
                               rpy=(0,np.pi/2,0), color=SURF_COLOR, parent="sofa_left"
                              )

sofa_right_front = gscene.create_safe(GEOTYPE.BOX, "sofa_right_front", link_name="base_link",
                               dims=(sofa_right.dims[2]-SOFA_MARGIN_Iz, sofa_right.dims[1])+(CLEARANCE,), 
                               center=(sofa_right.dims[0]/2+SOFA_UV_DIST,0,+SOFA_MARGIN_Iz/2), collision=False,# (오른쪽앞 바닥 오프셋)
                               rpy=(0,np.pi/2,0), color=SURF_COLOR, parent="sofa_right"
                              )

sofa_back_front = gscene.create_safe(GEOTYPE.BOX, "sofa_back_front", link_name="base_link",# (등앞 바닥 오프셋, 등앞 좌우 오프셋)
                               dims=(max(TOOL_DIM), sofa_back.dims[1]-SOFA_SIDE_THIC*2-SOFA_MARGIN_Iy*2)+(CLEARANCE,),
#                                dims=(sofa_back.dims[2]-SOFA_DIMS[2]-SOFA_MARGIN_I, sofa_back.dims[1]-SOFA_MARGIN_I*3)+(CLEARANCE,), 
                               center=(sofa_back.dims[0]/2+SOFA_UV_DIST,0,sofa_back.dims[2]/2-sofa.dims[2]/2+max(TOOL_DIM)/2+SOFA_MARGIN_Iz),#  (등앞 아래 오프셋)
                               rpy=(0,np.pi/2,0), color=SURF_COLOR, collision=False, parent="sofa_back"
                              )

In [9]:
# Qlist = []

In [10]:
# Qlist.append(crob.get_real_robot_pose())

In [11]:
# for iq, Q in enumerate(Qlist):
#     gscene.add_highlight_axis("hl", "pt{}".format(iq),
#                              T=brush_face.get_tf_handle(Q),
#                              dims=(0.3,0.03,0.03))

In [12]:
# save_pickle("teach_points.pkl", Qlist)

In [13]:
ADD_POLES = False
if ADD_POLES:
    pole_pt_list = kmm.remove_poles_by_box(gscene, gscene.NAME_DICT["room_ws"], 
                        pole_pt_list, Qcur, inside=False, margin=0.1)
    pole_pt_list = kmm.remove_poles_by_box(gscene, gscene.NAME_DICT["table_col"], 
                        pole_pt_list, Qcur, margin=0.2)
    pole_pt_list = kmm.remove_poles_by_box(gscene, gscene.NAME_DICT["sofa_col"], 
                        pole_pt_list, Qcur, margin=0.2)
    # pole_pt_list = kmm.remove_poles_by_box(gscene, gscene.NAME_DICT["sofa_back_col"], 
    #                     pole_pt_list, Qcur, margin=0.1)
    # pole_pt_list = kmm.remove_poles_by_box(gscene, gscene.NAME_DICT["sofa_right_col"], 
    #                     pole_pt_list, Qcur, margin=0.1)

    pole_list = kmm.add_pixel_poles("obs_pt", gscene, pole_pt_list, pole_res, height=WS_HEIGHT)
    gcheck.ignore_always = pole_list

    gscene.update_markers_all() 

### Set UI

In [14]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


### 1.0 Wait task start queue

In [15]:
if CONNECT_TASK_PLANNER:
    while servicer.object_info_running.object_id < 0:
        time.sleep(1)

   Use a production WSGI server instead.



## 2. Prepare cleaning

In [16]:
from pkg.planning.constraint.constraint_common import *
from pkg.planning.constraint.constraint_actor import *
from pkg.planning.constraint.constraint_subject import *
from pkg.utils.code_scraps import get_look_motion

 * Debug mode: off


In [17]:
# mplan.reset_log(flag_log=True)
Qcur = crob.get_real_robot_pose()
Qcur[-1] = np.deg2rad(-45)
Qcur[6:] = crob.home_pose[6:]
crob.joint_move_make_sure(Qcur)
Qcur = crob.get_real_robot_pose()
if CONNECT_INDY:
    indy.ktool.led_off()

End up at=[ 0.049  0.174  0.024] (3.47e-18 / 3.47e-18)


In [19]:
# mplan.reset_log(flag_log=True)
Qcur = crob.get_real_robot_pose()
Qcur[-1] = np.deg2rad(-45)
if not CONNECT_INDY:
    Qcur[6:] = crob.home_pose[6:]
crob.joint_move_make_sure(Qcur)
HOME_POSE_SWEEP = np.copy(Qcur)
# HOME_POSE_SWEEP[6:] = 0
crob.home_pose = HOME_POSE_SWEEP
crob.home_dict = list2dict(crob.home_pose, 
                           gscene.joint_names)
floor_ws = gscene.NAME_DICT["floor_ws"]    

VEL_LIMS = 0.4
ACC_LIMS = 0.4
SWP_VEL_LIMS = 0.2
SWP_ACC_LIMS = 0.2
RADI_DEG = 1

if CONNECT_INDY:
    indy.QVEL_LEVEL = 1
#     indy.collision_policy = indy_trajectory_client_nosdk.POLICY_NO_COLLISION_DETECTION
    indy.reset()

gtimer = GlobalTimer()
    
mode_switcher=ModeSwitcherLED(pscene, robot_name=ROBOT_NAME, brush_face=brush_face.geometry)

def no_offset(gxter, crob, mplan, robot_name, Qref):
    return Qref, Qref

def cleaning_fun(surface, tip_dir, sweep_dir, tool_dir, Qcur,
                 swp_vel_lims=SWP_VEL_LIMS, swp_acc_lims=SWP_ACC_LIMS,
                 div_num=None):
    if sweep_dir.lower() == "z":
        EE_HEIGHT = None
    else:
        T_e_brush = brush_face.get_tf_handle(crob.home_dict, from_link=TIP_LINK)
        T_brush_e = SE3_inv(T_e_brush)
        EE_HEIGHT = round(surface.get_tf(HOME_DICT)[2,3] + surface.dims[2]/2, 5) \
                        + T_brush_e[2, 3] - INDY_BASE_OFFSET[2]
    gtimer.reset(scale=1, timeunit='s', stack=True)
    gxter = GreedyExecuter(ppline, brush_face, TOOL_DIM, Qcur, 
                           vel_lims=VEL_LIMS, acc_lims=ACC_LIMS, 
                           swp_vel_lims=swp_vel_lims, swp_acc_lims=swp_acc_lims
                          )

    gxter.set_test_kwargs(multiprocess=PLANNING_MULTIPROC, N_agents=N_AGENTS,
                          timeout=TIMEOUT_MOTION, timeout_loop=TIMEOUT_FULL, 
                          verbose=VERBOSE, max_solution_count=MAX_SOL_NUM)

    gxter.get_division_dict(surface, tip_dir, sweep_dir, EE_HEIGHT,div_num=div_num)
    gxter.init_base_divs(Qcur)
    # gxter.mark_tested(None, None, covereds_all, [])
    snode_schedule_list, Qcur, covereds = gxter.greedy_execute(
        Qcur, tool_dir, mode_switcher, no_offset, cost_cut=BASE_COST_CUT)

    gxter.test_clear()
    print(gtimer)
    gscene.clear_virtuals()
    return snode_schedule_list, Qcur

End up at=[ 0.049  0.174  0.024] (3.47e-18 / 3.47e-18)


## 3. Table cleaning

In [23]:
snode_schedule_list_table_all = [snode_schedule_list_table]

In [22]:
snode_schedule_list_table_all = []

In [20]:
snode_schedule_list_table, Qcur = cleaning_fun(surface=table_surf, tip_dir="front", sweep_dir="X", tool_dir=1, Qcur=Qcur,
                                              div_num=(4,4))
snode_schedule_list_table_all.append(snode_schedule_list_table)

('Height Reference: ', 0.45399999845027922)
Q: [ 0.17  0.09 -0.    0.    0.    0.    0.    0.87 -1.22 -0.   -1.57  0.
 -0.79]
val: 103.0 / 110.0
End up at=[ 0.175  0.085 -0.   ] (0.0 / 0.0)
[PLAN] Adjust base once. ((0.995, -0.035000000000000003, -0.73099999999999998), (0.0, 0.0, 1.0, 0.0)) / ((1.0, -0.040000000000000001, -0.72999999999999998), (1.0, 0.0))
[PLAN] Qcur: [ 0.175  0.085 -0.   ]
[PLAN] Qref: [ 0.175  0.085 -0.   ]
[PLAN] tar: [ 0.175  0.085 -0.   ]
End up at=[ 0.175  0.085 -0.   ] (0.0 / 0.0)
[PLAN] Line idc [0, 4, 8, 12]
[PLAN] Try idc (0, 4, 8, 12)
Use 10/12 agents
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
======================= terminated 5: required answers acquired  (0.5/5.0) ===============================
Goal reached
Goal reached
======================= terminated 0: required answers acquired  (0.6/5.0) ===============================
======================= terminated 9: required answers acquired  (0.6/5.0) ====================

Goal reached
======================= terminated 5: required answers acquired  (0.5/5.0) ===============================
======================= terminated 7: Stop called from other agent  (0.5/5.0) ===============================
======================= terminated 2: Stop called from other agent  (0.5/5.0) ===============================
Goal reached
Goal reached
======================= terminated 1: required answers acquired  (0.6/5.0) ===============================
Goal reached
======================= terminated 3: required answers acquired  (0.6/5.0) ===============================
======================= terminated 0: required answers acquired  (0.6/5.0) ===============================
Goal reached
======================= terminated 4: required answers acquired  (0.6/5.0) ===============================
Goal reached
======================= terminated 6: required answers acquired  (0.6/5.0) ===============================
========================== FINISHED (0.6 / 5.0 s) ==========

NameError: name 'snode_schedule_list_table_all' is not defined

## 4. Sofa cleaning

### 4.1 upside

In [26]:
snode_schedule_list_sofa_all = []

In [27]:
snode_schedule_list_sofa, Qcur = cleaning_fun(surface=sofa_surf, tip_dir="front", sweep_dir="X", tool_dir=1, Qcur=Qcur)
snode_schedule_list_sofa_all.append(snode_schedule_list_sofa)

('Height Reference: ', 0.34299999845027923)
Q: [ 0.05  1.7  -0.    0.    0.    0.    0.    0.87 -1.22  0.   -1.57  0.
 -0.79]
val: 102.0 / 110.0
Send to mobile: [ 0.064  1.679  0.013  1.   ]
             -> [ 0.053  1.705 -0.     1.   ]
Distance=0.0389270999113 ([-0.011  0.026 -0.027])
[WARN] TOO SMALL VARIANCE, REDUCE TIMEOUT to 4.87
goal reach: 1 -> 0 (1640136136.2)
goal reach: 0 -> 1 (1640136136.9)
goal reach signal edge up
End up at=[ 0.062  1.713  0.033] (0.035 / 0.035)
[PLAN] Adjust base once. ((0.997, -0.30499999999999999, -0.62), (0.0, 0.0, 1.0, 0.0)) / ((1.0, -0.29999999999999999, -0.62), (1.0, 0.0))
[PLAN] Qcur: [ 0.053  1.705 -0.   ]
[PLAN] Qref: [ 0.053  1.705 -0.   ]
[PLAN] tar: [ 0.053  1.705 -0.   ]
Send to mobile: [ 0.062  1.713  0.016  1.   ]
             -> [ 0.053  1.705 -0.     1.   ]
Distance=0.0349628162683 ([-0.009 -0.008 -0.033])
[WARN] TOO SMALL VARIANCE, REDUCE TIMEOUT to 4.37
[WARN] ROBOT MOTION TIMEOUT
End up at=[ 0.062  1.713  0.033] (0.035 / 0.035)
[PLAN] 

LED ON
[KTOOL] led_on
[KTOOL] recv [162, 6, 0, 0]
Connect: Server IP (192.168.0.3)
LED OFF
[KTOOL] led_off
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)



In [28]:
# snode_schedule_list_sofa, Qcur = cleaning_fun(surface=sofa_back_surf, tip_dir="front", sweep_dir="X", tool_dir=1, Qcur=Qcur)
# snode_schedule_list_sofa_all.append(snode_schedule_list_sofa)

In [29]:
# snode_schedule_list_sofa, Qcur = cleaning_fun(surface=sofa_right_surf, tip_dir="front", sweep_dir="X", tool_dir=1, Qcur=Qcur)
# snode_schedule_list_sofa_all.append(snode_schedule_list_sofa)

In [30]:
# snode_schedule_list_sofa, Qcur = cleaning_fun(surface=sofa_left_surf, tip_dir="front", sweep_dir="X", tool_dir=1, Qcur=Qcur)
# snode_schedule_list_sofa_all.append(snode_schedule_list_sofa)

In [31]:
snode_schedule_list_sofa, Qcur = cleaning_fun(surface=sofa_back_front, tip_dir="down", sweep_dir="Z", tool_dir=-1, Qcur=Qcur)
snode_schedule_list_sofa_all.append(snode_schedule_list_sofa)

Q: [ 0.07  1.67 -0.    0.    0.    0.    0.    0.87 -1.22  0.   -1.57  0.
 -0.79]
val: 102.0 / 110.0
Send to mobile: [ 0.062  1.713  0.016  1.   ]
             -> [ 0.071  1.675 -0.     1.   ]
Distance=0.0511426890166 ([ 0.009 -0.038 -0.033])
goal reach: 1 -> 0 (1640136202.2)
goal reach: 0 -> 1 (1640136203.4)
goal reach signal edge up
End up at=[ 0.061  1.683  0.023] (0.0264 / 0.0264)
[PLAN] Adjust base once. ((0.73999999999999999, -0.27500000000000002, 1.179), (0.70699999999999996, -0.0, -0.70699999999999996, 0.0)) / ((0.73999999999999999, -0.27000000000000002, 1.1799999999999999), (0.70999999999999996, 0.0))
[PLAN] Qcur: [ 0.071  1.675 -0.   ]
[PLAN] Qref: [ 0.071  1.675 -0.   ]
[PLAN] tar: [ 0.071  1.675 -0.   ]
Send to mobile: [ 0.061  1.683  0.011  1.   ]
             -> [ 0.071  1.675 -0.     1.   ]
Distance=0.0264393817345 ([ 0.01  -0.008 -0.023])
[WARN] TOO SMALL VARIANCE, REDUCE TIMEOUT to 3.3
goal reach: 1 -> 0 (1640136205.1)
goal reach: 0 -> 1 (1640136205.9)
goal reach signa

Connect: Server IP (192.168.0.3)
LED ON
[KTOOL] led_on
[KTOOL] recv [162, 6, 0, 0]
Connect: Server IP (192.168.0.3)
LED OFF
[KTOOL] led_off
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)



In [23]:
# snode_schedule_list_sofa, Qcur = cleaning_fun(surface=sofa_right_front, tip_dir="down", sweep_dir="Z", tool_dir=-1, Qcur=Qcur)
# snode_schedule_list_sofa_all.append(snode_schedule_list_sofa)

In [24]:
# snode_schedule_list_sofa, Qcur = cleaning_fun(surface=sofa_front, tip_dir="down", sweep_dir="Z", tool_dir=-1, Qcur=Qcur)
# snode_schedule_list_sofa_all.append(snode_schedule_list_sofa)

Send to mobile: [  1.17400000e+00   6.92000000e-01   1.00000000e+00  -1.00000000e-03]
             -> [ 1.552  0.695  1.     0.   ]
Distance=0.377883404328 ([ 0.378  0.003 -0.002])
goal reach: 1 -> 0 (1639993992.43)
goal reach: 0 -> 1 (1639994002.03)
goal reach signal edge up
End up at=[ 1.565  0.728  3.137] (0.0357 / 0.0357)
[PLAN] Adjust base once. ((0.27500000000000002, -0.0050000000000000001, 1.107), (0.70699999999999996, -0.0, -0.70699999999999996, 0.0)) / ((0.28000000000000003, -0.0, 1.1100000000000001), (0.70999999999999996, 0.0))
[PLAN] Qcur: [ 1.552  0.695  3.142]
[PLAN] Qref: [ 1.552  0.695  3.142]
[PLAN] tar: [ 1.552  0.695  3.142]
Send to mobile: [ 1.565  0.728  1.     0.002]
             -> [ 1.552  0.695  1.     0.   ]
Distance=0.0357052677701 ([-0.013 -0.033  0.005])
[WARN] TOO SMALL VARIANCE, REDUCE TIMEOUT to 4.46
goal reach: 1 -> 0 (1639994003.53)
goal reach: 0 -> 1 (1639994005.13)
goal reach signal edge up
End up at=[ 1.544  0.694 -3.137] (6.28 / 6.28)
[PLAN] Line id

======================= terminated 1: node queue empty 3th  (1.5/5.0) ===============================
======================= terminated 2: node queue empty 4th  (1.5/5.0) ===============================
======================= terminated 7: node queue empty 2th  (1.5/5.0) ===============================
======================= terminated 3: node queue empty 5th  (1.6/5.0) ===============================
======================= terminated 4: node queue empty 6th  (1.6/5.0) ===============================
======================= terminated 6: node queue empty 7th  (1.6/5.0) ===============================
======================= terminated 8: node queue empty 8th  (1.6/5.0) ===============================
======================= terminated 0: node queue empty 9th  (1.7/5.0) ===============================
======================= terminated 9: node queue empty 10th  (1.7/5.0) ===============================
========================== FINISHED (1.8 / 5.0 s) ==============================]

======================= terminated 4: node queue empty 8th  (3.0/5.0) ===============================
======================= terminated 2: node queue empty 9th  (3.0/5.0) ===============================
======================= terminated 9: node queue empty 10th  (3.0/5.0) ===============================
========================== FINISHED (3.0 / 5.0 s) ==============================]
[PLAN] Try idc (5,)
Use 10/12 agents
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
======================= terminated 3: required answers acquired  (0.9/5.0) ===============================
Goal reached
======================= terminated 4: required answers acquired  (0.9/5.0) ===============================
======================= terminated 0: Stop called from other agent  (1.0/5.0) ===============================
======================= terminated 1: Stop called from other agent  (1.0/5.0) ===============================
Goal reached
Goal reached
=======================

Move Tool: -44.5083193145 (1639994019.9)
Connect: Server IP (192.168.0.3)
Move Tool: -44.0166386291 (1639994020.1)
Connect: Server IP (192.168.0.3)
Move Tool: -43.0332772582 (1639994020.3)
Connect: Server IP (192.168.0.3)
Move Tool: -42.5415965727 (1639994020.5)
Connect: Server IP (192.168.0.3)
Move Tool: -41.5582352018 (1639994020.7)
Connect: Server IP (192.168.0.3)
Move Tool: -41.0665545163 (1639994020.9)
Connect: Server IP (192.168.0.3)
Move Tool: -40.5748738309 (1639994021.1)
Connect: Server IP (192.168.0.3)
Move Tool: -40.0831931454 (1639994021.3)
Connect: Server IP (192.168.0.3)
Move Tool: -39.3319619114 (1639994021.5)
Connect: Server IP (192.168.0.3)
Move Tool: -38.5807306774 (1639994021.7)
Connect: Server IP (192.168.0.3)
Move Tool: -37.8294994434 (1639994021.9)
Connect: Server IP (192.168.0.3)
Move Tool: -37.0782682094 (1639994022.1)
Connect: Server IP (192.168.0.3)
Move Tool: -36.3270369754 (1639994022.3)
Connect: Server IP (192.168.0.3)
Move Tool: -35.5758057414 (1639994022.

Move Tool: -47.6004517674 (1639994042.37)
Connect: Server IP (192.168.0.3)
Move Tool: -47.28204534 (1639994042.57)
Connect: Server IP (192.168.0.3)
Move Tool: -46.9280553719 (1639994042.77)
Connect: Server IP (192.168.0.3)
Move Tool: -46.5387208295 (1639994042.97)
Connect: Server IP (192.168.0.3)
Move Tool: -46.33088698 (1639994043.17)
Connect: Server IP (192.168.0.3)
Move Tool: -45.8890686745 (1639994043.37)
Connect: Server IP (192.168.0.3)
Move Tool: -45.6551518924 (1639994043.57)
Connect: Server IP (192.168.0.3)
Move Tool: -45.4077348761 (1639994043.77)
Connect: Server IP (192.168.0.3)
Move Tool: -45.4077348761 (1639994043.98)
Connect: Server IP (192.168.0.3)
Move Tool: -45.4077348761 (1639994044.18)
Connect: Server IP (192.168.0.3)
Move Tool: -45.6551518924 (1639994044.38)
Connect: Server IP (192.168.0.3)
Move Tool: -45.6551518924 (1639994044.58)
Connect: Server IP (192.168.0.3)
Move Tool: -45.8890686745 (1639994044.78)
Connect: Server IP (192.168.0.3)
Move Tool: -46.1143268644 (16

Move Tool: -47.9823340935 (1639994065.65)
Connect: Server IP (192.168.0.3)
Move Tool: -47.056757169 (1639994065.85)
Connect: Server IP (192.168.0.3)
Move Tool: -46.1311802445 (1639994066.05)
Connect: Server IP (192.168.0.3)
Move Tool: -45.2056033201 (1639994066.25)
Connect: Server IP (192.168.0.3)
Move Tool: -44.2800263956 (1639994066.45)
Connect: Server IP (192.168.0.3)
Move Tool: -43.3544494711 (1639994066.65)
Connect: Server IP (192.168.0.3)
Move Tool: -43.3544494711 (1639994066.85)
Connect: Server IP (192.168.0.3)
Move Tool: -43.3544494711 (1639994067.05)
Connect: Server IP (192.168.0.3)
Move Tool: -43.3544494711 (1639994067.24)
Connect: Server IP (192.168.0.3)
LED ON
[KTOOL] led_on
[KTOOL] recv [162, 6, 0, 0]
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Move Tool: -43.3544487099 (1639994068.25)
Connect: Server IP (192.168.0.3)
Move Tool: -43.3544487099 (1639994068.45)
Connect: Server IP (192.168

Move Tool: -45.5808566613 (1639994088.53)
Connect: Server IP (192.168.0.3)
Move Tool: -45.5808566613 (1639994088.72)
Connect: Server IP (192.168.0.3)
Move Tool: -45.5546702767 (1639994088.92)
Connect: Server IP (192.168.0.3)
Move Tool: -45.5284838922 (1639994089.12)
Connect: Server IP (192.168.0.3)
Move Tool: -45.5022975077 (1639994089.33)
Connect: Server IP (192.168.0.3)
Move Tool: -45.4761111232 (1639994089.53)
Connect: Server IP (192.168.0.3)
Move Tool: -45.4499247387 (1639994089.73)
Connect: Server IP (192.168.0.3)
Move Tool: -45.4237383542 (1639994089.93)
Connect: Server IP (192.168.0.3)
Move Tool: -45.3975519696 (1639994090.13)
Connect: Server IP (192.168.0.3)
Move Tool: -45.3713655851 (1639994090.33)
Connect: Server IP (192.168.0.3)
Move Tool: -45.3713655851 (1639994090.53)
Connect: Server IP (192.168.0.3)
Move Tool: -45.3451792006 (1639994090.73)
Connect: Server IP (192.168.0.3)
Move Tool: -45.3189928161 (1639994090.93)
Connect: Server IP (192.168.0.3)
Move Tool: -45.2928064316

Move Tool: -5.81772826978 (1639994108.57)
Connect: Server IP (192.168.0.3)
Move Tool: -6.24738611268 (1639994108.78)
Connect: Server IP (192.168.0.3)
Move Tool: -6.66341945848 (1639994108.97)
Connect: Server IP (192.168.0.3)
Move Tool: -7.46729147092 (1639994109.17)
Connect: Server IP (192.168.0.3)
Move Tool: -8.60061207056 (1639994109.38)
Connect: Server IP (192.168.0.3)
Move Tool: -9.64352128135 (1639994109.58)
Connect: Server IP (192.168.0.3)
Move Tool: -10.5920946892 (1639994109.78)
Connect: Server IP (192.168.0.3)
Move Tool: -11.4422956415 (1639994109.98)
Connect: Server IP (192.168.0.3)
Move Tool: -11.9523116548 (1639994110.18)
Connect: Server IP (192.168.0.3)
Move Tool: -12.189771021 (1639994110.38)
Connect: Server IP (192.168.0.3)
Move Tool: -12.189771021 (1639994110.58)
Connect: Server IP (192.168.0.3)
Move Tool: -11.9523116548 (1639994110.78)
Connect: Server IP (192.168.0.3)
Move Tool: -11.170083927 (1639994110.98)
Connect: Server IP (192.168.0.3)
Move Tool: -10.286626276 (16

In [25]:
indy.QVEL_LEVEL

1

In [26]:
indy.traj_vel

1

In [ ]:
# snode_schedule_list_sofa, Qcur = cleaning_fun(surface=sofa_left_front, tip_dir="down", sweep_dir="Z", tool_dir=-1, Qcur=Qcur)
# snode_schedule_list_sofa_all.append(snode_schedule_list_sofa)

In [32]:
indy.ktool.op_init()

[KTOOL] op_init
[KTOOL] recv [162, 6, 0, 0]


False

# Save schedule

In [24]:
save_schedules(snode_schedule_list_table_all, snode_schedule_list_sofa_all)

In [22]:
snode_schedule_list_table_all, snode_schedule_list_sofa_all = load_schedules()

[INFO] data_folder is not specified - select most recent data
schedule loaded from /home/kiro-ros/Projects/rnb-planning/data/demo1222/20211222-102714


In [32]:
for snode_schedule_list in snode_schedule_list_table_all:
    play_cleaning_schedule(ppline, table_surf, snode_schedule_list, mode_switcher, TOOL_DIM)

Q: [ 0.06  1.68  0.02  0.    0.    0.  ]
val: 102.0 / 50.0
[INFO] Depart via: [ 0.05  1.43  0.02] (81.0) <- [ 0.06  1.68  0.02] (102.0)
Send to mobile: [ 0.057  1.679  0.011  1.   ]
             -> [ 0.046  1.426  0.011  1.   ]
Distance=0.25371022815 ([-0.011 -0.253  0.   ])
goal reach: 1 -> 0 (1640136302.7)
goal reach: 0 -> 1 (1640136310.0)
goal reach signal edge up
End up at=[-0.013  1.415  0.051] (0.0661 / 0.0661)
Q: [ 0.15  0.21 -0.    0.    0.    0.  ]
val: 103.0 / 50.0
[INFO] Approach through: [ 0.03  0.36 -0.  ] (81.0) -> [ 0.15  0.21 -0.  ] (103.0)
Send to mobile: [-0.013  1.415  0.019  1.   ]
             -> [ 0.027  0.362 -0.     1.   ]
Distance=1.05429064807 ([ 0.041 -1.053 -0.039])
goal reach: 1 -> 0 (1640136311.6)
goal reach: 0 -> 1 (1640136326.3)
goal reach signal edge up
End up at=[ 0.121  0.302  0.025] (0.114 / 0.114)
Send to mobile: [ 0.121  0.302  0.012  1.   ]
             -> [ 0.153  0.207 -0.     1.   ]
Distance=1.47558741861 ([ 0.096 -1.472 -0.022])
goal reach: 1 

KeyboardInterrupt: 

In [ ]:
for snode_schedule_list in snode_schedule_list_sofa_all:
    play_cleaning_schedule(ppline, sofa_surf, snode_schedule_list, mode_switcher, TOOL_DIM)

In [31]:
def print_shcedule_list_info(name, snode_schedule_list_all):
    print("="*11+" "+name+" "+"="*11)
    print("{} face count: {}".format(name, len(snode_schedule_list_all)))
    for i_face, snode_schedule_list_face in enumerate(snode_schedule_list_all):
        print(" face {} location count: {}".format(i_face, len(snode_schedule_list_face)))
        for i_loc, snode_schedule_loc in enumerate(snode_schedule_list_face):
            print("  loc {} snode count: {}".format(i_loc, len(snode_schedule_loc)))
            for i_s, snode in enumerate(snode_schedule_loc):
                if snode.traj is not None:
                    print("  snode {} traj length: {}".format(i_s, len(snode.traj)))
    print("="*(24+len(name)))
    
def print_difference(Q, DQ):
    print("original: {}".format(np.round(brush_face.get_tf_handle(Q, from_link=ROBOT_BASE)[:3,3], 3)))
    print("changed : {}".format(np.round(brush_face.get_tf_handle(Q+DQ, from_link=ROBOT_BASE)[:3,3], 3)))

def print_traj_change(traj, DQ_i, DQ_e):
    print("init")
    print_difference(traj[0], DQ_i)
    print("end")
    print_difference(traj[-1], DQ_e)

def update_traj_start_end(traj, DQ_i, DQ_e):
    len_traj = len(traj)
    traj_add = np.multiply((np.arange(len_traj, dtype=float)/(len_traj-1))[:,np.newaxis],
                           np.subtract(DQ_e, DQ_i)[np.newaxis, :])+np.array(DQ_i)[np.newaxis,:]
    return np.add(traj, traj_add)

def print_traj_ie(traj):
    print("initial: {}".format(np.round(brush_face.get_tf_handle(traj[0], from_link=ROBOT_BASE)[:3,3], 3)))
    print("end    : {}".format(np.round(brush_face.get_tf_handle(traj[-1], from_link=ROBOT_BASE)[:3,3], 3)))
    
def plot_compare_traj(traj1, traj2):
    plt.figure(figsize=(15,4))
    plt.subplot(1,3,1)
    plt.title("original")
    plt.plot(traj1)
    plt.subplot(1,3,2)
    plt.title("changed")
    plt.plot(traj2)
    plt.subplot(1,3,3)
    plt.title("difference")
    plt.plot(traj2-traj1)

In [34]:
print_shcedule_list_info("sofa", snode_schedule_list_sofa_all)

=========== sofa ===========
sofa face count: 3
 face 0 location count: 1
  loc 0 snode count: 10
  snode 1 traj length: 34
  snode 2 traj length: 76
  snode 4 traj length: 11
  snode 5 traj length: 76
  snode 7 traj length: 16
  snode 8 traj length: 76
  snode 9 traj length: 30
 face 1 location count: 2
  loc 0 snode count: 10
  snode 1 traj length: 37
  snode 2 traj length: 56
  snode 4 traj length: 37
  snode 5 traj length: 126
  snode 7 traj length: 14
  snode 8 traj length: 126
  snode 9 traj length: 56
  loc 1 snode count: 4
  snode 1 traj length: 51
  snode 2 traj length: 56
  snode 3 traj length: 53
 face 2 location count: 2
  loc 0 snode count: 10
  snode 1 traj length: 58
  snode 2 traj length: 96
  snode 4 traj length: 16
  snode 5 traj length: 56
  snode 7 traj length: 6
  snode 8 traj length: 56
  snode 9 traj length: 47
  loc 1 snode count: 4
  snode 1 traj length: 45
  snode 2 traj length: 96
  snode 3 traj length: 51


In [18]:
play_cleaning_schedule(ppline, table_surf, snode_schedule_list_table, mode_switcher, TOOL_DIM)

Send to mobile: [ 0.171  0.022 -0.002  1.   ]
             -> [ 0.073  0.035 -0.     1.   ]
Distance=0.0989855288991 ([-0.098  0.013  0.004])
goal reach: 1 -> 0 (1640077360.66)
goal reach: 0 -> 1 (1640077364.06)
goal reach signal edge up
End up at=[ 0.074  0.044 -0.003] (0.00933 / 0.00933)
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
LED ON
[KTOOL] led_on
[KTOOL] recv [162, 6, 0, 0]
Connect: Server IP (192.168.0.3)
LED OFF
[KTOOL] led_off
Connect: Server IP (192.168.0.3)
LED ON
[KTOOL] led_on
[KTOOL] recv [162, 6, 0, 0]
Connect: Server IP (192.168.0.3)
LED OFF
[KTOOL] led_off
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Send to mobile: [ 0.074  0.044 -0.002  1.   ]
             -> [ 0.168  0.035 -0.     1.   ]
Distance=0.0945529685895 ([ 0.094 -0.009  0.003])
goal reach: 1 -> 0 (1640077450.37)
goal reach: 0 -> 1 (1640077452.76)
goal reach signal edge up
End up at=[ 0.175  0.034 -0.004] (0.0083 / 0.0083)
Connect:

In [32]:
(20-6)/2

7

In [75]:
play_cleaning_schedule(ppline, table_surf, snode_schedule_list_sofa, mode_switcher, TOOL_DIM)

Q: [-0.07 -0.14 -0.03  0.    0.    0.  ]
val: 103.0 / 50.0
[INFO] Depart via: [-0.02 -0.05 -0.03] (102.0) <- [-0.07 -0.14 -0.03] (103.0)
Send to mobile: [-0.067 -0.137 -0.015  1.   ]
             -> [-0.024 -0.052 -0.015  1.   ]
Distance=0.0946837630114 ([ 0.043  0.084  0.   ])
goal reach: 1 -> 0 (1640133669.91)
goal reach: 0 -> 1 (1640133672.51)
goal reach signal edge up
End up at=[-0.015 -0.032 -0.041] (0.0244 / 0.0244)
Q: [ 0.18  1.75 -0.    0.    0.    0.  ]
val: 105.2 / 50.0
[INFO] Approach through: [ 0.03  1.55 -0.  ] (90.0) -> [ 0.18  1.75 -0.  ] (105.199981689)
Send to mobile: [-0.015 -0.032 -0.02   1.   ]
             -> [ 0.034  1.555 -0.     1.   ]
Distance=1.58809772165 ([ 0.049  1.587  0.041])
goal reach: 1 -> 0 (1640133674.21)
goal reach: 0 -> 1 (1640133698.61)
goal reach signal edge up
End up at=[  1.00000000e-03   1.53700000e+00  -6.20000000e-02] (0.0716 / 0.0716)
Send to mobile: [  1.00000000e-03   1.53700000e+00  -3.10000000e-02   1.00000000e+00]
             -> [ 0.1

### Finish

In [28]:
if CONNECT_TASK_PLANNER:
    servicer.mark_task_finished()

* 맵 폴 추가 v
* 플래닝 마치고 동작 시 빨간 하이라이트 없애기 v
* 스케줄 세이브 / 로드 구현하기 v
* NoSDK 7DoF 구현하기 v

* no-sdk 버전 mode switcher에 닦기 속도 조절 구현하기
* sdk 버전 제어기 개선 - 아래쪽, 먼 곳 닦을 때 토크 과다?